In [1]:
!pip install PySastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 3.6 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import os
import pickle
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import string

#### Load saved models, vectorizer, and selector

In [2]:
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stop_factory = StopWordRemoverFactory()
stop_words = stop_factory.get_stop_words()

In [3]:
titles = ['backend', 'ai', 'devops', 'frontend', 'gamedev', 'ios', 'android']

In [4]:
file = open("/content/bernoulli_model_v8_c7_e255.pickle", "rb")
bernoulli = pickle.load(file)
file.close()

file = open("/content/mnb_model_v8_c7_e255.pickle", "rb")
multinomial = pickle.load(file)
file.close()

file = open("/content/logistic_model_v8_c7_e255.pickle", "rb")
logistic_regression = pickle.load(file)
file.close()

In [5]:
with open('/content/tfidf_vectorizer_v8.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/select_kbest_v8.pkl', 'rb') as f:
    selector = pickle.load(f)

#### Normalization and text preprocessing

In [6]:
class VoteClassifier():
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, prep_data):
        votes = [0] * len(titles)
        for c in self._classifiers:
            v = c.predict_proba(prep_data)
            votes += v
        votes /= len(self._classifiers)
        return votes


In [7]:
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

def normalize_negation(text):
    words = word_tokenize(text)
    negation = ['tidak', "bukan"]
    document = []
    i = 0
    while i < len(words):
        if any(string in words[i] for string in negation):
            antonym_found = False
            synset = wordnet.synsets(words[i+1])
            for s in synset:
                for synonym in s.lemmas():
                    for antonym in synonym.antonyms():
                        document.append(antonym.name())
                        antonym_found = True
                        break
                    if antonym_found == True:
                        break
                if antonym_found == True:
                    break
            i+=1
        else:
            document.append(words[i])
        i+=1
    return document

In [33]:
s = "saya senang dengan pemrograman yang berkaitan dengan server dan API"

In [34]:
data = [s]
prep_data = []
# data = normalize_negation(data)

for d in data:
    d = [stemmer.stem(dt).lower() for dt in d.split(' ')]
    d = [dt for dt in d if dt not in stop_words]
    prep_data.append(' '.join(d))

In [35]:
prep_data_tfidf = tfidf.transform(prep_data)

prep_data_selected = selector.transform(prep_data_tfidf)

prep_data_2d = prep_data_selected

In [36]:
classifier = VoteClassifier(bernoulli, multinomial, logistic_regression)

In [37]:
res = classifier.classify(prep_data_2d)
print(res)
print(titles[np.argmax(res)])

[[0.46169782 0.05350755 0.03731374 0.07746534 0.0734444  0.13062977
  0.16594139]]
backend
